In [1]:
# Package Setup
import Pkg;
# Pkg.update()

# Julia Packages
using Distributions
using LinearAlgebra
using Statistics
using PGFPlots

# Satellite Dynamics Packages
using SatelliteDynamics

# Load SatelliteTasking - Reclone to keep version current
Pkg.clone("..")
using SatelliteTasking

┌ Warning: Pkg.clone is only kept for legacy CI script reasons, please use `add`
└ @ Pkg.API /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Pkg/src/API.jl:386


 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /Users/deddy/.julia/compiled/v1.1/SatelliteTasking/HFrko.ji for SatelliteTasking [30f2e302-0ae1-11e9-1cda-0f4324764595]
└ @ Base loading.jl:1184
ERROR: LoadError: LoadError: ArgumentError: Package SatelliteTasking does not have Analysis in its dependencies:
- If you have SatelliteTasking checked out for development and have
  added Analysis as a dependency but haven't updated your primary
  environment's manifest file, try `Pkg.resolve()`.
- Otherwise you may need to report an issue with SatelliteTasking
Stacktrace:
 [1] require(::Module, ::Symbol) at ./loading.jl:836
 [2] include at ./boot.jl:326 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /Users/deddy/Stanford/repos/SatelliteTasking.jl/src/SatelliteTasking.jl:2
 [6] top-level scope at none:0
 [7] include at ./boot.jl:326 [inlined]
 [8] include_relative(::Module, ::String) at ./loading.jl:1038
 [9] inclu

ErrorException: Failed to precompile SatelliteTasking [30f2e302-0ae1-11e9-1cda-0f4324764595] to /Users/deddy/.julia/compiled/v1.1/SatelliteTasking/HFrko.ji.

In [2]:
# Configure Simulation - Epoch, True Orbit, Test Images

# Simulation Horizon - 1 Day
epc0 = Epoch(2019, 1, 1, 0, 0, 0, tsys=:UTC)
epcf = Epoch(2019, 1, 2, 0, 0, 0, tsys=:UTC)

oe  = [R_EARTH + 500e3, 0, 90.0, 0, 0, 0]
eci = sOSCtoCART(oe, use_degrees=true)
println("Initial Orbital Elements: $oe")
println("Initial Inertial State: $eci")

# Simulate Orbit 
@time orbit = Orbit(epc0, epcf, eci, dtmax=5.0);

# Load Test Images
@time images = load_images("../data/landsat_test.json");

Initial Orbital Elements: [6.87814e6, 0.0, 90.0, 0.0, 0.0, 0.0]
Initial Inertial State: [6.87814e6, 0.0, 0.0, -0.0, 4.66138e-13, 7612.61]


UndefVarError: UndefVarError: Orbit not defined

In [3]:
# Compute True Windows of Visibility
@time true_collects = find_all_collects(orbit, images, sort=true);

# # True Collect TImes:
# for col in collects
#     println(col)
# end

UndefVarError: UndefVarError: find_all_collects not defined

In [4]:
# Create normal distribution of position uncertainty

pos_std  = 5000.0/sqrt(3.0) # 5000 #1000 # 100
vel_std  = 5.0/sqrt(3.0) # 5 # 1 # 0.1
orb_mean = deepcopy(eci)
orb_cov  = diagm(0 => [pos_std^2, pos_std^2, pos_std^2, vel_std^2, vel_std^2, vel_std^2])
dist_eci = MvNormal(orb_mean, orb_cov)


FullNormal(
dim: 6
μ: [6.87814e6, 0.0, 0.0, -0.0, 4.66138e-13, 7612.61]
Σ: [8.33333e6 0.0 … 0.0 0.0; 0.0 8.33333e6 … 0.0 0.0; … ; 0.0 0.0 … 8.33333 0.0; 0.0 0.0 … 0.0 8.33333]
)


In [5]:
# Generate and compute statistics for multiple orbits

# Number of samples to simulate orbit uncertainty
N_SAMPLES = 3

orbits             = []
perturbed_collects = []
perturbed_stats    = []

@time begin
    for i in 1:N_SAMPLES
        @time begin
            println("Simulating orbit: $i")
            pert_eci = rand(dist_eci, 1)[:, 1]
            println("Initial ECI Error: $(pert_eci - eci)")
            
            # Simulate perturbed orbit
            pert_orb = Orbit(epc0, epcf, pert_eci, dtmax=5.0)
            push!(orbits, pert_orb)
            
            # Compute Collection times
            pert_collects = find_all_collects(orbits[end], images, sort=true)
            push!(perturbed_collects, pert_collects)
            
            # Compute Collection Statistics
            col_mean, col_sdev = collect_stats(true_collects, [pert_collects])
            push!(perturbed_stats, (col_mean, col_sdev))
        end
    end
end

Simulating orbit: 1
Initial ECI Error: [2729.81, 759.032, -130.073, 0.935604, -3.84891, -2.17681]


UndefVarError: UndefVarError: Orbit not defined

In [6]:
# Compute error in 24-hour bins
ms = hcat([stats[1] for stats in perturbed_stats]...)

0-element Array{Any,1}

In [7]:
sd = hcat([stats[2] for stats in perturbed_stats]...)

0-element Array{Any,1}

In [8]:
println("Mean difference in collect windows: $(transpose(ms))")
println("Standard Deviation of collect windows: $(transpose(sd))")

m_ms = mean(ms, dims=2)
println(m_ms)
m_sd = mean(sd, dims=2)
println(m_sd)

Mean difference in collect windows: 
Standard Deviation of collect windows: 


ArgumentError: ArgumentError: reducing over an empty collection is not allowed

In [9]:
# Analyze Collects on a per-satellite basis
hourly_stats = []
hourly_means = []
hourly_stds  = []

@time begin
for collects in perturbed_collects
    push!(hourly_means, [])
    push!(hourly_stds, [])
    for i in 0:23
        # Hourly bins of windows
        epc_window_start = epc0 + 3600*i
        epc_window_end   = epc0 + 3600*(i+1)
        
#         println("Analysis Window: $epc_window_start - $epc_window_end")

        m, s = collect_stats(true_collects, [collects], epc_min=epc_window_start, epc_max=epc_window_end)
        if m != nothing && s != nothing
            push!(hourly_means[end], m)
            push!(hourly_stds[end], s)
#           println(m, s)
        end
    end
end
end

hourly_means = [hcat(means...) for means in hourly_means]
hourly_stds  = [hcat(stds...) for stds in hourly_stds]

  0.000006 seconds


0-element Array{Any,1}

In [10]:
p = plot(1:24, hourly_stds[1][3, :]);

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1]

In [11]:
# Analyze Collects on a per-satellite basis
hourly_stats = []
hourly_means = []
hourly_stds  = []

@time begin
# Aggregate collects across all satellites simulated
collects = vcat(perturbed_collects...)
    
for i in 0:23
    # Hourly bins of windows
    epc_window_start = epc0 + 3600*i
    epc_window_end   = epc0 + 3600*(i+1)

#         println("Analysis Window: $epc_window_start - $epc_window_end")

    m, s = collect_stats(true_collects, [collects], epc_min=epc_window_start, epc_max=epc_window_end)
    if m != nothing && s != nothing
        push!(hourly_means, m)
        push!(hourly_stds, s)
#           println(m, s)
    end
end
end

hourly_means = hcat(hourly_means...)
hourly_stds  = hcat(hourly_stds...)

UndefVarError: UndefVarError: collect_stats not defined

In [12]:
Axis(Plots.Linear(1:24, hourly_stds[1, :]), width="10cm", height="10cm", xlabel="Hour", ylabel="Std of Collect Window Start Time [sec]")

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1, Base.Slice(Base.OneTo(1))]

In [13]:
Axis([
    Plots.Linear(1:24, hourly_stds[1, :], legendentry="Start of Window")
    Plots.Linear(1:24, hourly_stds[2, :], legendentry="End of Window")
    Plots.Linear(1:24, hourly_stds[3, :], legendentry="Window Duration")
], width="10cm", height="10cm", legendPos="north west")

BoundsError: BoundsError: attempt to access 0-element Array{Any,1} at index [1, Base.Slice(Base.OneTo(1))]